In [1]:
import pandas as pd
import numpy as np

In [2]:
legal_columns_to_display = [
    "Patient",
    "Nbre de lames",
    "Taille (cm)",
    "Valeur exacte AFP pré-opératoire",
    "Nombre de nodules",
    "Expansif multinodulaire",
    "Récidive avant 2 ans",
]

In [3]:
dfs = pd.read_excel("../data/tabs/table_prognosis.xlsx", sheet_name=None)

df_pb = dfs[list(dfs.keys())[0]][legal_columns_to_display]
df_hm = dfs[list(dfs.keys())[1]][legal_columns_to_display]
df_bj = dfs[list(dfs.keys())[2]][legal_columns_to_display]

temp = pd.concat([df_pb, df_hm], ignore_index=True)
df_clin = pd.concat([temp, df_bj], ignore_index=True)

df_clin[
    [
        "log1p_taille",
        "log1p_AFP",
    ]
] = df_clin[
    [
        "Taille (cm)",
        "Valeur exacte AFP pré-opératoire",
    ]
].apply(np.log1p)

df_clin.rename(columns={"Récidive avant 2 ans": "Récidive Globale"}, inplace=True)
df_clin.rename(columns={"Patient": "patient"}, inplace=True)

In [4]:
df_clin.head()

,patient,Nbre de lames,Taille (cm),Valeur exacte AFP pré-opératoire,Nombre de nodules,Expansif multinodulaire,Récidive Globale,log1p_taille,log1p_AFP
0,1,3.0,2.5,13.6,1.0,0.0,0.0,1.252763,2.681022
1,2,6.0,5.5,785.0,1.0,0.0,0.0,1.871802,6.666957
2,3,5.0,3.3,600.0,1.0,0.0,1.0,1.458615,6.398595
3,4,7.0,3.5,7.0,1.0,0.0,0.0,1.504077,2.079442
4,5,5.0,3.0,2.0,1.0,0.0,1.0,1.386294,1.098612


In [5]:
df_tumor = pd.read_csv("../data/tabs/final_tumor_features.csv")
df_nuclear = pd.read_csv("../data/tabs/final_nuclear_features.csv")
df_inflams = pd.read_csv("../data/tabs/final_inflams_features.csv")

temp1 = pd.merge(df_clin, df_tumor, on="patient", how="inner")
temp2 = pd.merge(df_nuclear, df_inflams, on="patient", how="inner")
data = pd.merge(temp1, temp2, on="patient", how="inner")
data.head()

,patient,Nbre de lames,Taille (cm),Valeur exacte AFP pré-opératoire,Nombre de nodules,Expansif multinodulaire,Récidive Globale,log1p_taille,log1p_AFP,%P,NP_CntArea_norm,P_CntArea_norm,%P_max,P_CntArea_norm_max,density,mean nucleus area,anisocaryose,nucleocyto index,peri-tumoral,intra-tumoral
0,36,8.0,7.3,10000.0,1.0,1.0,1.0,2.116256,9.210440,0.999418,0.012956,26.663323,1.000000,39.523915,209.732812,1104.475688,2814.687303,0.742903,34.352733,19.149136
1,63,4.0,3.5,24.0,1.0,1.0,0.0,1.504077,3.218876,1.000000,0.000000,24.921369,1.000000,24.921369,219.023052,127.822056,61.533426,0.284339,31.372811,10.456491
2,222,3.0,5.0,3.0,1.0,0.0,0.0,1.791759,1.386294,0.880749,0.482221,3.353707,0.995724,8.347394,148.792627,119.821466,46.042405,0.518789,5.305010,2.465104
3,111,3.0,7.0,11.0,1.0,0.0,0.0,2.079442,2.484907,0.091228,30.735117,0.484662,0.109476,0.839771,89.174804,170.353167,106.349919,0.062319,7.508969,5.738979
4,164,4.0,7.0,1.0,1.0,0.0,1.0,2.079442,0.693147,0.806528,0.280626,7.078524,0.910763,15.459250,176.194656,96.003088,34.950598,0.162315,3.057164,3.130673


In [ ]:
data.to_excel("../data/tabs/input_dataframe_prognosis.xlsx")